In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import glob
import sys
import dask

from CASutils import lensread_utils as lens
from CASutils import readdata_utils as read
from math import nan

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [ ]:
topdir="/glade/campaign/cesm/collections/CESM2-SF/timeseries/"

In [ ]:
landfrac = xr.open_dataset('LANDFRAC_LENS2.nc')
landfrac = landfrac.landfrac

In [ ]:
landfrac = xr.where(landfrac > 0.5, 1, nan)

In [ ]:
memstr = [ str(imem).zfill(3) for imem in np.arange(1,16,1) ]

In [16]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '50GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='02:00:00',
    interface='ib0')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [17]:
cluster

PBSCluster('tcp://10.12.206.54:35957', workers=15, threads=15, memory=322.12 GB)

In [8]:
#client.close()

In [18]:
client

Client Scheduler: tcp://10.12.206.54:35957 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Cluster Workers: 15 Cores: 15 Memory: 322.12 GB


In [19]:
for imem in memstr:
    print(imem)
    filelist = sorted(glob.glob(topdir+"b.e21.B1850cmip6.f09_g17.CESM2-SF-AAER."+imem+
                                "/lnd/proc/tseries/day_1/*.FSNO.*.nc")+
                glob.glob(topdir+'b.e21.B1850cmip6.f09_g17.CESM2-SF-AAER-SSP370.'+imem+
                                "/lnd/proc/tseries/day_1/*.FSNO.*.nc"))
    dat = xr.open_mfdataset(filelist, concat_dim=['time'], coords='minimal')[['FSNO','time_bounds']]
    dat = read.fixcesmtime(dat, timebndsvar='time_bounds')
    dat['lon'] = landfrac.lon ; dat['lat'] = landfrac.lat
    dat = dat.sel(time=slice("1850-01","2050-12"))
    #dat = dat.where( ((dat.time.dt.month == 5) | (dat.time.dt.month == 6) | (dat.time.dt.month == 7)), drop=True)
    dat = dat.where( ((dat.time.dt.month == 5) ), drop=True)
    #datmasked = dat.FSNO*landfrac
    datmasked = dat.FSNO.where(dat.lat > 50, drop=True).load()
    #datmasked_stacked = datmasked.stack(z=("lon","lat"))
    #datmasked_stacked = datmasked_stacked.dropna("z")
    #datmasked_stacked = datmasked_stacked.load()
    #datmasked_stacked = datmasked_stacked.reset_index("z").load()
    datmasked.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/FSNO/FSNO_MAY_50N90N_AAER2_"+imem+"_1850to2050.nc")

001
002
003
004
005
006
007
008
009
010
011
012
013
014
015


In [20]:
cluster.close()

In [17]:
cluster.close()